<a href="https://colab.research.google.com/github/achett/Hierarchical-Model/blob/main/Bayesian_Hierarchical_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hierarchicalforecast
!pip install statsforecast
!pip install datasetsforecast
!pip install nixtlats>=0.1.0
!pip install darts
!pip install mlforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 113.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.2/508.2 kB 64.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.85 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 893.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 18.1 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=e6025e937f84674a5d73969254c8b8a40b34d6d4ede5e287f4cd705ce862cac4
  Stored in directory: /root/.cache/pip/wheels/5a/b5/4e/216953a1c711da55de29ed7ecf158b4a5bf32ef93d69ad66dd
Successfully built fugue-s

In [160]:
########################
# PACKAGES
########################
# !pip install -U numba statsforecast datasetsforecast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime
import pymc as pm

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive, AutoETS, AutoCES, AutoTheta

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, OptimalCombination, ERM
from hierarchicalforecast.utils import aggregate

# #obtain hierarchical dataset
# from datasetsforecast.hierarchical import HierarchicalData
from nixtlats import TimeGPT
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.dataprocessing.transformers import Scaler
from lightgbm import LGBMRegressor
from darts.metrics import mae, rmse
from darts.models import LightGBMModel
from darts import concatenate
# Import necessary libraries
import pandas as pd
from darts import TimeSeries
from darts.models import ExponentialSmoothing, StatsForecastAutoETS, StatsForecastAutoARIMA, KalmanForecaster
from darts.metrics import mae, mape

import pandas as pd
from darts import TimeSeries
from darts.models import RegressionModel
from darts.dataprocessing.transformers import Scaler
from lightgbm import LGBMRegressor
from darts.models import LightGBMModel
from darts import concatenate
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from statsmodels.tsa.stattools import adfuller


pd.options.display.float_format = '{:,.2f}'.format

In [405]:
##############
# PARAMS
##############
fct_periods = 12
fct_st_date = '2023-04-01'
fct_end_date = '2023-12-01'

# Create hierarchical structure and constraints
hierarchy_levels = [['top_level'],
                    ['top_level', 'level2'],
                    ['top_level', 'level2', 'level3'],
                    ['top_level', 'level2', 'level3', 'bottom_level']]

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/regional_hierarchy.csv'
r_hier = pd.read_csv(inputFile)


In [162]:
##############
# FUNCTIONS
##############

def prepare_data(data, r_hier):
    # Merge hierarchy
    data = data.merge(r_hier, how='left', left_on='cost_object', right_on='cost_object')

    # Transform date and y
    data['ds'] = pd.to_datetime(data['ds'])
    data['y'] = data['y'].astype(float)

    # Address NA values
    data['volume'] = data['y'].fillna(0)
    data['region'] = data['region'].fillna('')
    data['cost_object'] = data['cost_object'].fillna('')
    data['product'] = data['product'].fillna('')

    # Create hierarchical dataframe
    data['top_level'] = 'global'  # Assuming 'top_level' does not contain '/', otherwise add a replace line for it too.
    data.rename(columns={'region': 'level2', 'cost_object': 'level3', 'product': 'bottom_level'}, inplace=True)
    data = data[['level2', 'level3', 'bottom_level', 'top_level', 'ds', 'y']]

    # Replace '/' with '_' in the four columns
    data['level2'] = data['level2'].str.replace('/', '_')
    data['level3'] = data['level3'].str.replace('/', '_')
    data['bottom_level'] = data['bottom_level'].str.replace('/', '_')

    data['unique_id'] = data['top_level'] + '/' + data['level2'] + '/' + data['level3'] + '/' + data['bottom_level']

    return data

def convert_fct2df(forecasts):
    forecast_dfs = []
    for unique_id, forecast_ts in forecasts.items():
        df = forecast_ts.pd_dataframe()
        df['unique_id'] = unique_id
        forecast_dfs.append(df)

    # Concatenate all forecast DataFrames into a single DataFrame
    all_forecasts_df = pd.concat(forecast_dfs, axis=0)
    all_forecasts_df.reset_index(inplace=True)

    # Reorder and rename columns as needed
    all_forecasts_df = all_forecasts_df[['unique_id', 'ds', 'y']]
    all_forecasts_df.columns.name = None

    return all_forecasts_df

def prepare_feature(data, r_hier, volume_act2, feature_name):

    # Select and rename columns
    data = data[['cost_object', 'product', 'ds', feature_name]].rename(columns={feature_name: 'y'})

    # Apply any additional preparation (assuming prepare_data is a function you have defined)
    data = prepare_data(data, r_hier)

    # Rename the columns back
    data = data.rename(columns={'y': feature_name})

    # Merge with the volume_act2 dataframe
    merged_df = data.merge(volume_act2[['unique_id', 'ds']], how='right', on=['unique_id', 'ds'])

    return merged_df


In [434]:
##############
# DATA LOAD
##############
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/budgetFY23.csv'
budget = pd.read_csv(inputFile)
budget = budget[budget['category']=='EQUIV_UNIT - Equivalent Units']
budget.rename(columns={'country': 'cost_object'}, inplace=True)
budget = prepare_data(budget, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_output.csv'
volume_act = pd.read_csv(inputFile)
volume_act.rename(columns={'value': 'y'}, inplace=True)
volume_act = prepare_data(volume_act, r_hier)

inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
sga = pd.read_csv(inputFile)

sga1 = prepare_feature(sga, r_hier, volume_act, 'AP')
sga2 = prepare_feature(sga, r_hier, volume_act, 'Field_Sales')


inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/ids2smooth.csv'
ids2smooth = pd.read_csv(inputFile)

In [435]:
volume_act['level2'].unique()

array(['Greater China', 'Japan', 'East Europe', 'BENELUX',
       'North America', 'West Europe', 'Nordic', 'Baltic', 'MEA', 'LATAM',
       'SESA', 'CIS'], dtype=object)

In [436]:
########################
# SAMPLE
########################
# Subset
# regs2include = ['US10 - Astellas Pharma US, Inc.', 'US21 - Agensys, Inc.', 'JP10 - Astellas Pharma Inc']
# volume_act = volume_act[volume_act['level3'].isin(regs2include)]

# level2include = ['CIS']
# volume_act = volume_act[volume_act['level2'].isin(level2include)]

In [437]:
########################
# IDENTIFY UNIVERSE
########################
tested_ts = set(budget['unique_id'].unique()).intersection(volume_act['unique_id'].unique())

# Find unique IDs present in budget_h but not in rev
unique_ids_in_budget_not_in_rev = set(budget['unique_id'].unique()).difference(volume_act['unique_id'].unique())

# Find unique IDs present in rev but not in budget_h
unique_ids_in_rev_not_in_budget = set(volume_act['unique_id'].unique()).difference(budget['unique_id'].unique())

# Filter volume
volume_act = volume_act[volume_act['unique_id'].isin(tested_ts)]

In [438]:
########################
# INTERMITTENT DEMAND CANDIDATES
########################

# Function to calculate the percentage of zeros after the first non-zero
def calculate_percentage_zeros(df):
    # Find the index of the first non-zero entry in 'y'
    first_non_zero_index = df.loc[df['y'] != 0].index.min()
    # If there are no non-zero values, return None or 0 based on your preference
    if pd.isna(first_non_zero_index):
        return None  # Or return 0 if you want to treat this as 0% zeros following non-zero
    # Select the subset of 'y' after the first non-zero
    post_non_zero_series = df.loc[first_non_zero_index:, 'y']
    # Count the number of zeros in this subset
    num_zeros = (post_non_zero_series == 0).sum()
    # Calculate the percentage of zeros
    percentage_zeros = num_zeros / len(post_non_zero_series) * 100
    return percentage_zeros

# Apply the function to each group and reset index to make unique_id a column
percentage_zeros_df = volume_act.groupby('unique_id').apply(calculate_percentage_zeros).reset_index(name='percentage_zeros')

percentage_zeros_df


,unique_id,percentage_zeros
0,global/BENELUX/D_E_BELGIUM - Belgium/ENFORTUMA...,0.00
1,global/BENELUX/D_E_BELGIUM - Belgium/ENZA - En...,0.00
2,global/BENELUX/D_E_BELGIUM - Belgium/GILTERITN...,0.00
3,global/BENELUX/D_E_BELGIUM - Belgium/MIRABEGRO...,0.00
4,global/BENELUX/D_E_BELGIUM - Belgium/SOLIFENAC...,0.00
...,...,...
584,global/West Europe/D_E_SWITZ - Switzerland/ROX...,NaN
585,global/West Europe/D_E_SWITZ - Switzerland/SOL...,0.00
586,global/West Europe/D_E_SWITZ - Switzerland/TAC...,0.00
587,global/West Europe/D_E_SWITZ - Switzerland/TAC...,0.00


In [439]:
########################
# CANDIDATES FOR TIME SERIES MODELS
########################
# Function to test stationarity with handling for constant series
def test_stationarity(timeseries):
    # Check if the series is constant
    if np.all(timeseries == timeseries.iloc[0]):
        return 'constant'  # Return 'constant' if all values in the series are the same

    # Perform Dickey-Fuller test otherwise:
    dftest = adfuller(timeseries, autolag='AIC')
    # Return True if series is stationary, False otherwise
    return dftest[1] <= 0.05

# Function to difference the series
def difference_series(timeseries):
    return timeseries.diff().dropna()

# Your original dataframe
df = volume_act

# Prepare a DataFrame to store the results
results_df = pd.DataFrame(columns=['unique_id', 'original_stationary', 'differenced_stationary'])

# Loop through each unique_id
for unique_id in df['unique_id'].unique():
    # Extract the time series for the current unique_id
    sub_series = df[df['unique_id'] == unique_id].set_index('ds')['y']

    # Test for stationarity on the original series
    original_stationary = test_stationarity(sub_series)

    # Initialize differenced_stationary as None (it will stay None if differencing is not needed)
    differenced_stationary = None

    # If the original series is not stationary, difference the series and test again
    if not original_stationary:
        differenced_series = difference_series(sub_series)
        differenced_stationary = test_stationarity(differenced_series)

    # Append the results to the results DataFrame
    results_df = results_df.append({'unique_id': unique_id,
                                    'original_stationary': original_stationary,
                                    'differenced_stationary': differenced_stationary},
                                   ignore_index=True)

stationary_list1 = results_df[(results_df['original_stationary']==True)]['unique_id'].tolist()
stationary_list2 = results_df[(results_df['original_stationary']==True) | (results_df['differenced_stationary']==True)]['unique_id'].tolist()

<ipython-input-439-63aa0682494e>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'unique_id': unique_id,
<ipython-input-439-63aa0682494e>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'unique_id': unique_id,
<ipython-input-439-63aa0682494e>:42: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = results_df.append({'unique_id': unique_id,
<ipython-input-439-63aa0682494e>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'unique_id': unique_id,
<ipython-input-439-63aa0682494e>:42: FutureWarni

In [440]:
########################
# SEGMENT TIME SERIES
########################
new_products = ['ENFORTUMAB - Enforumab Vedotin', 'ROXADUSTNT - Roxadustant']
loe_products = ['REGADENOSN - Regadenoson']
div_products = ['MICAFUNGIN - Micafungin Sodium']

new_ids = volume_act[volume_act['bottom_level'].isin(new_products)]['unique_id'].unique().tolist()
loe_ids = volume_act[volume_act['bottom_level'].isin(loe_products)]['unique_id'].unique().tolist()
divested_ids = volume_act[volume_act['bottom_level'].isin(div_products)]['unique_id'].unique().tolist()

# IDs with A&P and Field Sales Spend
grouped1 = sga1.groupby('unique_id')[['AP']].sum()
grouped2 = sga2.groupby('unique_id')[['Field_Sales']].sum()
spend_ids = set(grouped1[(grouped1['AP'] > 0)].index.tolist() + grouped2[(grouped2['Field_Sales'] > 0)].index.tolist())
spend_ids = spend_ids.difference(new_ids + loe_ids + divested_ids)


arima_regions = ['Japan', 'BENELUX', 'North America', 'Nordic', 'Baltic']
ets_regions = ['East Europe', 'West Europe', 'Greater China', 'MEA', 'LATAM', 'SESA', 'CIS']
inter_demand_list = percentage_zeros_df[percentage_zeros_df['percentage_zeros']>=50]['unique_id'].tolist()
arima_ids = volume_act[(volume_act['level2'].isin(arima_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()
ets_ids = volume_act[(volume_act['level2'].isin(ets_regions)) & (~volume_act['unique_id'].isin(spend_ids))]['unique_id'].unique().tolist()

In [441]:
########################
# SMOOTH DATA
########################
# import numpy as np
# import pandas as pd
# import statsmodels.api as sm

# # Filter
# volume_act_sm = volume_act[volume_act['unique_id'].isin(ids2smooth['unique_id'])]

# # Columns to smooth
# columns_to_smooth = ['y']

# # New column names for the smoothed data
# new_columns = ['Smoothed_y']

# # Iterate through each unique ID
# for unique_id in volume_act_sm['unique_id'].unique():
#     # Filter the DataFrame for the current ID
#     temp_df = volume_act_sm[volume_act_sm['unique_id'] == unique_id]

#     # Iterate through each column to smooth
#     for col, new_col in zip(columns_to_smooth, new_columns):
#         # Ensure the time series is a pandas Series (assuming 'ds' is the datetime column)
#         time_series = pd.Series(temp_df[col].values, index=pd.to_datetime(temp_df['ds']))

#         # Set up and fit the model
#         model = sm.tsa.UnobservedComponents(time_series, 'local linear trend')
#         print(unique_id)
#         results = model.fit()

#         # Get the smoothed series
#         smoothed_series = results.smoothed_state[0]

#         # Ensure smoothed values are non-negative
#         smoothed_series = np.maximum(smoothed_series, 0)

#         # Adjust the smoothed series to maintain the original sum
#         original_sum = time_series.sum()
#         smoothed_sum = smoothed_series.sum()
#         if smoothed_sum > 0:  # Prevent division by zero
#             adjustment_factor = original_sum / smoothed_sum
#             adjusted_smoothed_series = smoothed_series * adjustment_factor
#         else:
#             adjusted_smoothed_series = smoothed_series  # Keep original if smoothed sum is zero

#         # Store the adjusted smoothed series back in the DataFrame
#         # Note: We're using 'temp_df.index' to ensure the indices match
#         volume_act_sm.loc[temp_df.index, new_col] = adjusted_smoothed_series

# # After looping, 'sga_output2' will contain the new adjusted smoothed columns
# volume_act_sm = volume_act_sm[['unique_id', 'ds', 'Smoothed_y']]
# volume_act_sm.columns = ['unique_id', 'ds', 'y']

# # Re-add non smoothed
# volume_act_smx = volume_act[~(volume_act['unique_id'].isin(ids2smooth['unique_id'])) & (volume_act['unique_id'].isin(budget['unique_id'])) & (volume_act['unique_id'].isin(tested_ts))]

# # Concat
# volume_act2 = pd.concat([volume_act_smx, volume_act_sm])

In [442]:
########################
# RUN AUTO ARIMA
########################
# # volume_act2 = volume_act[volume_act['unique_id'].isin(auto_arima_list)]
# volume_act2 = volume_act

# #split train/test sets
# test  = volume_act2.groupby('unique_id').tail(fct_periods)
# train = volume_act2.drop(test.index)

# models=[AutoETS(season_length=12), AutoARIMA(season_length = 12), AutoTheta(season_length = 12)]
# # models=[AutoARIMA(season_length = 12)]
# # models=[AutoETS(season_length=12)]

# # Compute base predictions
# fcst = StatsForecast(df=train[['unique_id', 'ds', 'y']],
#                      models=models,
#                      freq='MS', n_jobs=-1)

# volume_fct = fcst.forecast(h=fct_periods)

# volume_fct['Forecast'] = volume_fct[['AutoETS', 'AutoARIMA', 'AutoTheta']].mean(axis=1)
# volume_fct = volume_fct[['unique_id', 'ds', 'Forecast']]

In [443]:
########################
# RUN ETS
########################

def generate_time_series_dict(data, fct_periods, filter_data):
    # Split train/test sets
    test = data.groupby('unique_id').tail(fct_periods)
    train = data.drop(test.index)

    # Prepare time series dataframes
    time_series_dfs = {uid: group for uid, group in train.groupby('unique_id')}
    time_series_dict = {}

    if filter_data:
        # Filter out time series with insufficient non-zero data points
        filtered_time_series_dfs = {}
        for uid, group in time_series_dfs.items():
            non_zero_index = group['y'].ne(0).idxmax()
            start_index = max(0, non_zero_index - (13 - 1))
            filtered_df = group.loc[non_zero_index:] if group.loc[non_zero_index:].shape[0] >= 13 else group.loc[start_index:]
            if not filtered_df.empty:
                filtered_time_series_dfs[uid] = filtered_df
        # Convert each filtered DataFrame into a Darts TimeSeries object
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in filtered_time_series_dfs.items()}
    else:
        # Convert each original DataFrame into a Darts TimeSeries object without filtering
        time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in time_series_dfs.items()}

    return time_series_dict

def generate_forecast(data, fct_periods, model2use, filter_data=True):

    # Use the nested function to generate the time series dictionary
    time_series_dict = generate_time_series_dict(data, fct_periods, filter_data)

    # Create and fit a model for each time series
    models = {}
    for uid, series in time_series_dict.items():
        model = get_model(model2use)
        model.fit(series)
        models[uid] = model

    # Forecasting
    forecasts = {uid: model.predict(fct_periods) for uid, model in models.items()}

    # Convert forecasts into a dataframe
    # Assuming convert_fct2df is a function defined elsewhere
    fct = convert_fct2df(forecasts)

    return fct

# Function to dynamically get the model instance
def get_model(model_name):
    if model_name == 'AutoETS':
        return StatsForecastAutoETS()
    elif model_name == 'ARIMA':
        return StatsForecastAutoARIMA(season_length=12)
    elif model_name == 'KF':
        return KalmanForecaster(dim_x=12)
    else:
        raise ValueError(f"Unsupported model: {model_name}")

ets_fct = generate_forecast(volume_act, fct_periods, model2use='AutoETS', filter_data=True)
arima_fct = generate_forecast(volume_act, fct_periods, model2use='ARIMA', filter_data=True)
# kf_fct = generate_forecast(volume_act, fct_periods, model2use='KF', filter_data=True)

In [444]:
########################
# RUN ENSEMBLE
########################
# from darts.models import RegressionEnsembleModel, NaiveSeasonal, LinearRegressionModel
# import pandas as pd
# from darts import TimeSeries
# from darts.models import ExponentialSmoothing, StatsForecastAutoETS, StatsForecastAutoTheta, StatsForecastAutoARIMA
# from darts.metrics import mae, mape

# train2 = train[train['unique_id'].isin(['global/Japan/JP10 - Astellas Pharma Inc/ENZA - Enzalutamide', 'global/North America/US10 - Astellas Pharma US, Inc./ENZA - Enzalutamide', 'global/West Europe/D_E_FRANCE - France/ROXADUSTNT - Roxadustant'])]

# time_series_dfs = {uid: group for uid, group in train.groupby('unique_id')}

# # Convert each filtered DataFrame into a Darts TimeSeries object
# time_series_dict = {uid: TimeSeries.from_dataframe(group, 'ds', 'y') for uid, group in time_series_dfs.items()}

# # Create and fit model for each time series
# ensemble_models = {}

# for uid, series in time_series_dict.items():
#     model = RegressionEnsembleModel(forecasting_models = [StatsForecastAutoETS(season_length=12), StatsForecastAutoARIMA(season_length=12)],regression_train_n_points=36)

#     model.fit(series)
#     ensemble_models[uid] = model

# # Forecasting (assuming you want to forecast the next N steps)
# N = 12
# forecasts = {uid: model.predict(N) for uid, model in ensemble_models.items()}

# ensemble_fct = convert_fct2df(forecasts)

In [445]:
########################
# RUN QUARTERLY MODEL
########################
# Function to resample and sum data by quarter for each group
def resample_group(group):
    group = group.set_index('ds')  # Set 'ds' as the index
    resampled_group = group.resample('Q').agg({'y': 'sum'})  # Aggregate data by quarter
    return resampled_group

quarterly_data = volume_act
quarterly_data['ds'] = pd.to_datetime(quarterly_data['ds'])

# Group by 'unique_id' and apply the resampling function
quarterly_data = quarterly_data.groupby('unique_id').apply(resample_group).reset_index()



In [446]:
########################
# FEATURE ENGINEERING
########################


# Feature Engineering
inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_output.csv'
features = pd.read_csv(inputFile)

inputFile = '/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/price_output.csv'
price = pd.read_csv(inputFile)

inputFile = '/content/drive/MyDrive/Colab Notebooks/SGA Prediction/data/sga_output.csv'
sga = pd.read_csv(inputFile)

sga1 = prepare_feature(sga, r_hier, volume_act, 'AP')
sga2 = prepare_feature(sga, r_hier, volume_act, 'Field_Sales')
sga3 = prepare_feature(sga, r_hier, volume_act, 'Smoothed_AP')
sga4 = prepare_feature(sga, r_hier, volume_act, 'Smoothed_Field_Sales')

price1 = prepare_feature(price, r_hier, volume_act, 'Price')
price2 = prepare_feature(price, r_hier, volume_act, 'Smoothed_Price')


features = features[['cost_object', 'product', 'ds', 'value']]
features.rename(columns={'value': 'y'}, inplace=True)
features = prepare_data(features, r_hier)
features = features.merge(volume_act[['unique_id', 'ds']], how='right', on=['unique_id', 'ds'])

# Encoding 'cost_object' and 'product'
label_encoder = LabelEncoder()
features['region'] = label_encoder.fit_transform(features['level2'])
features['cost_object'] = label_encoder.fit_transform(features['level3'])
features['product'] = label_encoder.fit_transform(features['bottom_level'])
features = features[['unique_id', 'ds', 'region', 'cost_object', 'product', 'top_level']]

<ipython-input-446-c55051f0adc1>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.rename(columns={'value': 'y'}, inplace=True)


In [447]:
########################
# RUN LGBM
########################
time_series_dict = generate_time_series_dict(volume_act, fct_periods, filter_data=False)

# Prepare individual time series for dynamic features
dynamic_features_dict = {}
for uid in sga1['unique_id'].unique():
    df1 = sga1[sga1['unique_id'] == uid].sort_values('ds')
    df2 = sga2[sga2['unique_id'] == uid].sort_values('ds')
    df3 = sga3[sga3['unique_id'] == uid].sort_values('ds')
    df4 = sga4[sga4['unique_id'] == uid].sort_values('ds')
    df5 = price1[price1['unique_id'] == uid].sort_values('ds')
    df6 = price2[price2['unique_id'] == uid].sort_values('ds')

    # Create a TimeSeries object for features
    ts1 = TimeSeries.from_dataframe(df1[['ds', 'AP']], 'ds', 'AP')
    ts2 = TimeSeries.from_dataframe(df2[['ds', 'Field_Sales']], 'ds', 'Field_Sales')
    ts3 = TimeSeries.from_dataframe(df3[['ds', 'Smoothed_AP']], 'ds', 'Smoothed_AP')
    ts4 = TimeSeries.from_dataframe(df4[['ds', 'Smoothed_Field_Sales']], 'ds', 'Smoothed_Field_Sales')
    ts5 = TimeSeries.from_dataframe(df5[['ds', 'Price']], 'ds', 'Price')
    ts6 = TimeSeries.from_dataframe(df6[['ds', 'Smoothed_Price']], 'ds', 'Smoothed_Price')

    # Concatenate the two TimeSeries along the feature axis (axis=1)
    combined_ts = concatenate([ts1, ts2, ts3, ts6], axis=1)
    # combined_ts = concatenate([ts3], axis=1)

    # Store the combined TimeSeries in the dictionary
    dynamic_features_dict[uid] = combined_ts



# Create and train a regression model for each time series
models = {}
scalers = {}
forecasts = {}
for uid, ts_combined in time_series_dict.items():
    # Scale the combined series
    scaler = Scaler()
    ts_combined_transformed = scaler.fit_transform(ts_combined)
    scalers[uid] = scaler  # Store scaler for later inverse transformation

    # Initialize and fit your LightGBM model
    model = LightGBMModel(lags=len(ts_combined) // 2, output_chunk_length=1, verbose=-1, lags_past_covariates=12)
    model.fit(series=ts_combined_transformed, past_covariates=dynamic_features_dict[uid])

    # model = LightGBMModel(lags=len(ts_combined) // 2, output_chunk_length=1, verbose=-1)
    # model.fit(series=ts_combined_transformed)

    models[uid] = model

    # Generate forecasts
    forecast = model.predict(n=fct_periods, series=ts_combined_transformed)
    forecasts[uid] = scalers[uid].inverse_transform(forecast)  # Inverse transform to get back to original scale


# Convert forecasts to df
lgbm_fct = convert_fct2df(forecasts)

In [448]:
########################
# XTREND - DECAY
########################


def apply_exponential_decay(df, start_date, end_date, end_value_percentage, target_unique_ids):
    # Convert 'ds' to datetime if it's not already
    df['ds'] = pd.to_datetime(df['ds'])
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Function to apply decay for each group
    def decay_group(group):
        # Only apply changes if unique_id is in target_unique_ids
        if group['unique_id'].iloc[0] not in target_unique_ids:
            return group

        # Sort by date to ensure proper indexing
        group = group.sort_values(by='ds')

        # Initialize end_value
        end_value = None

        # Find start and end values and dates
        if start_date in group['ds'].values and end_date in group['ds'].values:
            start_value = group.loc[group['ds'] == start_date, 'y'].iloc[0]
            end_value = start_value * end_value_percentage

            # Calculate the decay rate based on exponential decay formula
            days = (end_date - start_date).days
            decay_rate = np.log(end_value / start_value) / days

            # Apply exponential decay for dates between start_date and end_date
            for date in pd.date_range(start_date, end_date):
                if date in group['ds'].values:
                    t = (date - start_date).days
                    new_value = start_value * np.exp(decay_rate * t)
                    group.loc[group['ds'] == date, 'y'] = new_value

        # Replace 'y' values for dates after end_date with the value at end_date
        if end_value is not None:  # Ensure there was an end value calculated
            group.loc[group['ds'] > end_date, 'y'] = end_value

        return group

    # Apply the decay to each group
    updated_df = df.groupby('unique_id').apply(decay_group).reset_index(drop=True)
    return updated_df

# Apply exponential decay
lgbm_fct.rename(columns={'LGBM': 'y'}, inplace=True)
ets_fct.rename(columns={'ETS': 'y'}, inplace=True)
arima_fct.rename(columns={'ARIMA': 'y'}, inplace=True)

# Micafungin
arima_fct = apply_exponential_decay(arima_fct, '2023-07-01', '2023-08-01', 0, divested_ids)
ets_fct = apply_exponential_decay(ets_fct, '2023-07-01', '2023-08-01', 0, divested_ids)

# Lexiscan
arima_fct = apply_exponential_decay(arima_fct, '2023-04-01', '2023-12-01', .1, loe_ids)
ets_fct = apply_exponential_decay(ets_fct, '2023-04-01', '2023-12-01', .1, loe_ids)


<ipython-input-448-680c715e1e2e>:31: RuntimeWarning: divide by zero encountered in log
  decay_rate = np.log(end_value / start_value) / days
<ipython-input-448-680c715e1e2e>:37: RuntimeWarning: invalid value encountered in scalar multiply
  new_value = start_value * np.exp(decay_rate * t)
<ipython-input-448-680c715e1e2e>:31: RuntimeWarning: invalid value encountered in scalar divide
  decay_rate = np.log(end_value / start_value) / days
<ipython-input-448-680c715e1e2e>:47: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  updated_df = df.groupby('unique_id').apply(decay_group).reset_index(drop=True)
<ipython-input-448-680c715e1e2e>:31: RuntimeWa

In [449]:
########################
# XTREND - GROWTH
########################
volume_act = volume_act[~volume_act['unique_id'].isin(new_ids)]

In [458]:
########################
# METRICS
########################


# Subset
volume_act_xsm = volume_act[['unique_id', 'ds', 'y']]
budget2 = budget[['unique_id', 'ds', 'y']]

# Assign names
volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
budget2.rename(columns={'y': 'Budget'}, inplace=True)
lgbm_fct.rename(columns={'y': 'LGBM'}, inplace=True)
ets_fct.rename(columns={'y': 'ETS'}, inplace=True)
arima_fct.rename(columns={'y': 'ARIMA'}, inplace=True)

# Merge actuals, budget and forecast
rev_at = volume_act_xsm.merge(ets_fct, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(budget2, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(lgbm_fct, on=['unique_id', 'ds'], how='left')
rev_at = rev_at.merge(arima_fct, on=['unique_id', 'ds'], how='left')


non_spend_ids = rev_at[~rev_at['unique_id'].isin(spend_ids)]['unique_id'].unique()

# Conditions for selection
conditions = [rev_at['unique_id'].isin(spend_ids),rev_at['unique_id'].isin(arima_ids),rev_at['unique_id'].isin(ets_ids)]

# Values corresponding to each condition
choices = [rev_at['LGBM'], rev_at['ARIMA'],rev_at['ETS']]

# Creating the new column 'SelectedFCT' based on the conditions
rev_at['SelectedFCT'] = np.select(conditions, choices, default=np.nan)




# Only keep tested ts
rev_at = rev_at[rev_at['unique_id'].isin(tested_ts)]

rev_at = rev_at[rev_at['unique_id'].isin(volume_act['unique_id'].unique())]




# Filter for dates
data4metrics = rev_at[(rev_at['ds']<=fct_end_date) & (rev_at['ds']>=fct_st_date)]

# Sum up the values for each unique_id
numeric_cols = data4metrics.columns.drop(['unique_id', 'ds'])
summed_df = data4metrics.groupby('unique_id')[numeric_cols].sum()

# Calculate difference and percentage differences from 'Actuals'
absolute_diff = summed_df.subtract(summed_df['Actuals'], axis=0).abs()
percentage_diff = summed_df.subtract(summed_df['Actuals'], axis=0).div(summed_df['Actuals'], axis=0).abs()

# Drop the 'Actuals' column as we don't need to compare it with itself
absolute_diff.drop(columns=['Actuals', 'ARIMA', 'LGBM', 'ETS'], inplace=True)

# Find the column with the lowest difference for each unique_id and add to metrics table
min_diff_col = absolute_diff.idxmin(axis=1)
data4metrics['lowest_diff_col'] = data4metrics['unique_id'].map(min_diff_col)

# Find winner
winner = data4metrics[data4metrics['unique_id'].isin(non_spend_ids)].groupby('lowest_diff_col')

# Get Budget winners
bud_winners = winner.get_group('Budget')['unique_id'].unique()
# ets_winners = winner.get_group('ETS')['unique_id'].unique()
# arima_winners = winner.get_group('ARIMA')['unique_id'].unique()
# lgbm_winners = winner.get_group('LGBM')['unique_id'].unique()


winner['unique_id'].nunique()

<ipython-input-458-23b382f2b55d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_act_xsm.rename(columns={'y': 'Actuals'}, inplace=True)
<ipython-input-458-23b382f2b55d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  budget2.rename(columns={'y': 'Budget'}, inplace=True)
<ipython-input-458-23b382f2b55d>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4metrics['low

lowest_diff_col
Budget         174
SelectedFCT    141
Name: unique_id, dtype: int64

In [451]:
# ets_winners2 = set(ets_winners).difference(set(arima_winners)).difference(set(lgbm_winners))

# arima_winners2 = set(arima_winners).difference(set(ets_winners)).difference(set(lgbm_winners))

# lgbm_winners2 = set(lgbm_winners).difference(set(arima_winners)).difference(set(ets_winners))

In [459]:
########################
# CREATE PLOT DATA
########################
fct_st_date = pd.to_datetime(fct_st_date)

# Add revenue actuals
data2plot = rev_at.copy()
data2plot['ds'] = pd.to_datetime(data2plot['ds'])

# Update Actuals columns
data2plot['Actuals (Train)'] = data2plot['Actuals'].copy()
data2plot['Actuals'] = data2plot.apply(lambda row: row['Actuals'] if row['ds'] >= fct_st_date else None, axis=1)
data2plot['Actuals (Train)'] = data2plot.apply(lambda row: row['Actuals (Train)'] if row['ds'] < fct_st_date else None, axis=1)

# Filter to end date
data2plot = data2plot[data2plot['ds']<=fct_end_date]

# Find TS to fix
ts2fix = data2plot[data2plot['unique_id'].isin(bud_winners)]
tsnonspend = data2plot[data2plot['unique_id'].isin(non_spend_ids)]

data2plot.head()

,unique_id,ds,Actuals,ETS,Budget,LGBM,ARIMA,SelectedFCT,Actuals (Train)
0,global/Greater China/D_CN_TOTAL - China Total/...,2014-04-01,NaN,NaN,NaN,NaN,NaN,NaN,0.00
1,global/Greater China/D_CN_TOTAL - China Total/...,2014-05-01,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2,global/Greater China/D_CN_TOTAL - China Total/...,2014-06-01,NaN,NaN,NaN,NaN,NaN,NaN,0.00
3,global/Greater China/D_CN_TOTAL - China Total/...,2014-07-01,NaN,NaN,NaN,NaN,NaN,NaN,0.00
4,global/Greater China/D_CN_TOTAL - China Total/...,2014-08-01,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [453]:
########################
# DECOMPOSITION
########################
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose, STL

def decompose_series(df, frequency='M'):
    # Create an empty DataFrame to store results
    results_df = pd.DataFrame()

    # Iterate through each unique_id in the DataFrame
    for unique_id in df['unique_id'].unique():
        # Filter the DataFrame for the unique_id
        group = df[df['unique_id'] == unique_id]

        # Convert 'ds' to datetime if not already and sort values
        group = group.sort_values('ds')

        # Find the first non-zero 'y' value
        start_index = group.loc[group['y'] > 0].index.min()

        # Subset the DataFrame from the first non-zero 'y'
        if pd.notnull(start_index):
            subset = group.loc[start_index:]

            # Resample and fill missing values if necessary
            subset.set_index('ds', inplace=True)
            subset_resampled = subset.resample(frequency).mean(numeric_only=True).ffill()

            # Determine the period for decomposition based on the number of observations
            if len(subset_resampled) < 24:
                period = 6  # Ensure period is odd for STL, adjust if necessary
            else:
                period = 12  # Ensure period is odd for STL, adjust if necessary

            # Make period odd for STL if it's not
            if period % 2 == 0:
                period += 1

            # Apply traditional decomposition if there are enough data points
            if len(subset_resampled) >= 2 * period:  # Ensure enough data for traditional decomposition
                traditional_decomp = seasonal_decompose(subset_resampled['y'], model='additive', period=max(period, 2))
                subset_resampled['traditional_trend'] = traditional_decomp.trend
                subset_resampled['traditional_seasonal'] = traditional_decomp.seasonal
                subset_resampled['traditional_residual'] = traditional_decomp.resid

            # Apply STL decomposition if there are enough data points
            if len(subset_resampled) >= 2 * period:  # Ensure enough data for STL decomposition
                stl = STL(subset_resampled['y'], seasonal=period, robust=True)  # Adjust parameters as needed
                stl_result = stl.fit()
                subset_resampled['stl_trend'] = stl_result.trend
                subset_resampled['stl_seasonal'] = stl_result.seasonal
                subset_resampled['stl_residual'] = stl_result.resid

            # Reset index to move 'ds' from index back to column
            subset_resampled.reset_index(inplace=True)
            subset_resampled['unique_id'] = unique_id  # Add the unique_id column to the subset

            # Concatenate the results
            results_df = pd.concat([results_df, subset_resampled], ignore_index=True)

    return results_df

# Apply the function to your DataFrame
decomposed_results = decompose_series(volume_act, frequency='M')  # You can change the frequency as needed
decomposed_results.head()


,ds,y,traditional_trend,traditional_seasonal,traditional_residual,stl_trend,stl_seasonal,stl_residual,unique_id
0,2020-02-29,797.00,NaN,"7,037.22",NaN,-255.10,812.98,239.12,global/Greater China/D_CN_TOTAL - China Total/...
1,2020-03-31,132.00,NaN,"-3,042.78",NaN,-121.62,"1,403.75","-1,150.13",global/Greater China/D_CN_TOTAL - China Total/...
2,2020-04-30,120.00,NaN,"-2,728.71",NaN,11.61,-243.66,352.05,global/Greater China/D_CN_TOTAL - China Total/...
3,2020-05-31,168.00,NaN,"1,337.95",NaN,145.35,-620.26,642.92,global/Greater China/D_CN_TOTAL - China Total/...
4,2020-06-30,582.00,NaN,997.14,NaN,280.33,223.13,78.54,global/Greater China/D_CN_TOTAL - China Total/...


In [462]:
########################
# PLOT
########################
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import base64
from io import BytesIO

# Update the function to include filtering based on 'unique_id'
def plot_data(unique_id):
    # Define x_column and y_columns directly
    x_column = data2use.columns[1]
    y_columns = [data2use.columns[2], data2use.columns[3], data2use.columns[4], data2use.columns[5], data2use.columns[6], data2use.columns[7], data2use.columns[8]]

    # Filter data based on selected unique_id
    filtered_data = data2plot[data2plot['unique_id'] == unique_id]

    # Set up a 1x3 grid of subplots
    fig, (ax1, ax4) = plt.subplots(1, 2, figsize=(25, 5), gridspec_kw={'width_ratios': [4, 1]}) # Adjust layout for table

    # Plotting multiple y-axes on the first subplot
    for y_column in y_columns:
        ax1.plot(filtered_data[x_column], filtered_data[y_column], label=y_column)
    ax1.set_xlabel(x_column)
    ax1.set_ylabel('Values')
    ax1.set_title(f'Revenue for {unique_id}')
    ax1.legend()

    # Remove axis for table
    ax4.axis('off')
    ax4.axis('tight')

    # Displaying the sum table
    display_data = filtered_data[[x_column] + list(y_columns)].copy()
    display_data = display_data[display_data['ds']>=fct_st_date]
    display_data['ds'] = display_data['ds'].dt.strftime('%m/%d/%Y')

    # Create a sum row
    sum_values = {x_column: 'Sum'}
    for col in list(y_columns):
        sum_values[col] = display_data[col].sum()
    sum_row = pd.DataFrame([sum_values])

    # Create a % diff row
    actuals_sum = sum_values['Actuals']
    pdiff_values = {x_column: '% Diff'}
    for col in list(y_columns):
        pdiff_values[col] = ((display_data[col].sum()-actuals_sum) / actuals_sum) * 100 if actuals_sum != 0 else None
        pdiff_values[col] = round(pdiff_values[col], 2)
    perc_diff_row = pd.DataFrame([pdiff_values])

    # Stack the sum row
    display_data = pd.concat([sum_row, display_data], ignore_index=True)

    # Round the values and add commas
    for column in y_columns:
        if column in display_data.columns:
            # Round to two decimal places
            display_data[column] = display_data[column].round(2)
            # Format with commas
            display_data[column] = display_data[column].apply(lambda x: f"{x:,.2f}")

    # Stack the % diff and remove 'Actuals Train'
    display_data = pd.concat([perc_diff_row, display_data], ignore_index=True)
    display_data = display_data.drop('Actuals (Train)', axis=1)

    # Convert perc_diff_data to array for table
    table_data = display_data.to_numpy()
    # Add table at the right
    table = ax4.table(cellText=table_data, colLabels=display_data.columns, loc='right')
    table.auto_set_font_size(False)
    table.set_fontsize(8.5)  # Set smaller font size if necessary
    table.scale(4, 1.8)  # Adjust scale to fit

    plt.tight_layout()
    plt.show()


data2use = ts2fix
# data2use = tsnonspend
# data2use = data2plot

# Create widgets
unique_id_selector = widgets.SelectionSlider(
    options=data2use['unique_id'].unique(),
    description='unique_id:',
    orientation='horizontal',
    readout=True
)

# Display interactive plot
interact(plot_data, unique_id=unique_id_selector)

interactive(children=(SelectionSlider(description='unique_id:', options=('global/Greater China/D_CN_TOTAL - Ch…

<function __main__.plot_data(unique_id)>

In [455]:
rev_at[~rev_at['unique_id'].isin(spend_ids)]['unique_id'].unique()

array(['global/Greater China/D_CN_TOTAL - China Total/ENZA - Enzalutamide',
       'global/Greater China/D_CN_TOTAL - China Total/FEBUXOSTAT - FEBUXOSTAT',
       'global/Greater China/D_CN_TOTAL - China Total/GILTERITNB - Gilteritinib',
       'global/Greater China/D_CN_TOTAL - China Total/MICAFUNGIN - Micafungin Sodium',
       'global/Greater China/D_CN_TOTAL - China Total/MIRABEGRON - Mirabegron',
       'global/Greater China/D_CN_TOTAL - China Total/OLOPATADIN - OLOPATADIN',
       'global/Greater China/D_CN_TOTAL - China Total/SOLIFENACN - Solifenacin Succinate',
       'global/Greater China/D_CN_TOTAL - China Total/TACROLIMUS - Tacrolimus',
       'global/Greater China/D_CN_TOTAL - China Total/TACRO_XL - Tacrolimus Extended-Release',
       'global/Greater China/D_CN_TOTAL - China Total/TAMSULOSIN - Tamsulosin HCl',
       'global/East Europe/D_E_AUSTRIA - Austria/MICAFUNGIN - Micafungin Sodium',
       'global/East Europe/D_E_AUSTRIA - Austria/SOLIF_TAMS - Solifenacin _ Tamsulo

In [456]:
####################
# CREATE FINAL DF AND SAVE DATA
####################
volume_fct = data4metrics[data4metrics['unique_id'].isin(tested_ts)]

volume_fct = volume_fct[['unique_id', 'ds', 'Forecast','Actuals', 'Budget', 'lowest_diff_col']]

# Sum up the values for each unique_id
numeric_cols = volume_fct.columns.drop(['unique_id', 'ds', 'lowest_diff_col'])
volume_sum = volume_fct.groupby('unique_id')[numeric_cols].sum()

# Calculate difference and percentage differences from 'Actuals'
absolute_diff = volume_sum.subtract(volume_sum['Actuals'], axis=0).abs()
percentage_diff = volume_sum.subtract(volume_sum['Actuals'], axis=0).div(volume_sum['Actuals'], axis=0).abs()

# Drop uneeded columns
absolute_diff.drop(columns=['Actuals', 'Budget'], inplace=True)

# Find the column with the lowest difference for each unique_id and add to metrics table
min_diff_col = absolute_diff.idxmin(axis=1)
volume_fct['lowest_diff_col'] = volume_fct['unique_id'].map(min_diff_col)

# Define a function to apply to each row
def get_value_from_column(row):
    return row[row['lowest_diff_col']]

# Apply this function to each row
volume_fct['fct'] = volume_fct.apply(get_value_from_column, axis=1)

# Select columns
volume_fct = volume_fct[['unique_id', 'ds','fct']]

volume_fct.to_csv('/content/drive/MyDrive/Colab Notebooks/Revenue Prediction/data/volume_fct.csv', index=False)

KeyError: "['Forecast'] not in index"